In [216]:
%load_ext autoreload
%autoreload 2
import numpy as np
out_dir = '../../results/nlp'
os.makedirs(out_dir, exist_ok=True)
from tqdm import tqdm
import torch
from pytorch_transformers import BertTokenizer, BertForMaskedLM, BertForSequenceClassification, BertConfig

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [217]:
from util import *
device = 'cpu'

In [218]:
default = 'bert-base-uncased'
mdir = '/scratch/users/vision/chandan/pacmed/glue/SST-2-3epoch' # '/scratch/users/vision/chandan/pacmed/glue/SST-2-middle/'

t = 'This great movie was very'
fill = 'good'

In [230]:
tokenizer = BertTokenizer.from_pretrained(mdir)
clf = BertForSequenceClassification.from_pretrained(mdir).eval().to(device)
masked_predictor = BertForMaskedLM.from_pretrained(default).eval().to(device)

**look at an individual prediction**

In [227]:
text = 'worst movie' #'amazing, wonderful, great, beautiful, gorgeous, incredible, best movie actress'
pred(text, tokenizer, clf)

0.0007611739

**predicted masked token**

In [231]:
text = f"[CLS] {t} {fill} . [SEP] I thoroughly enjoyed it [SEP]"
inds_max, preds_masked = predict_masked(masked_predictor,
                                        tokenizer,
                                        text=text,
                                        masked_index=6)

for ind in inds_max[:20]:
    print(f'{tokenizer.convert_ids_to_tokens([ind])[0]}\t{preds_masked[ind]:0.2f}')

entertaining	0.16
good	0.14
popular	0.06
successful	0.05
exciting	0.05
enjoyable	0.04
funny	0.04
interesting	0.03
satisfying	0.02
beautiful	0.02
special	0.02
fun	0.01
impressive	0.01
important	0.01
wonderful	0.01
emotional	0.01
moving	0.01
effective	0.01
engaging	0.01
original	0.01


**calculate predictions varying the masked token**

In [ ]:
n = 30522 # 30522
vals = np.zeros(n)
weights = np.zeros(n)
is_rand = np.random.choice(range(30522), size=n, replace=False)

for j in tqdm(range(n)):
    i = is_rand[j] # generate a random index
    sampled_word = inds_max[i]
    
    s = f'{t} {tokenizer.convert_ids_to_tokens([sampled_word])[0]}'
    vals[j] = pred(s, tokenizer, clf)
    weights[j] = preds_masked[sampled_word]
    
ave = np.mean(vals)
ave_weighted = np.average(vals, weights=weights)
pred_good = pred(f'{t} good', tokenizer, clf)
pred_bad = pred(f'{t} bad', tokenizer, clf)

In [235]:
print(f'ave={ave:0.3f} ave_weighted={ave_weighted:0.3f} pred_good={pred_good:0.3f} pred_bad={pred_bad:.3f}')

ave=0.813 ave_weighted=0.944 pred_good=0.997 pred_bad=0.003
